# Neural Network - Binary Classification

In [1]:
import pandas as pd
data = pd.read_csv('../../../datasets/breast_cancer_data.csv')
target = pd.read_csv('../../../datasets/breast_cancer_target.csv')

In [2]:
data.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,186.0000,275.0000,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,243.0000,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,173.00000
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,198.0000,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,205.0000,0.4000,0.1625,0.2364,0.07678


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.25)

In [4]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [5]:
# units = Quantos neurônios farão parte da camada oculta
# Para determinar o número de neurônios: (número de atributos previsores + número de classes)/ 2
# activation: função de ativação
# kernel_initializer: Incialização dos pesos
# input_dim = Quantos elementos na camada de entrada

classifier = Sequential()
# Camada oculta
classifier.add(Dense(units=16, activation='relu', kernel_initializer='random_uniform', input_dim=30))
# Camada oculta (o "input_dim" deve ser colocado apenas na primeira camada oculta)
classifier.add(Dense(units=16, activation='relu', kernel_initializer='random_uniform'))


# Camada de saída 
# Usamos a sigmoid, pois é um problema de classificação binária
classifier.add(Dense(units=1, activation='sigmoid'))

# Learnig Rate
# Decay: Quanto o learning rate será decrementado a cada iteração
# clipvalue: Uma forma de "prender" o valor para que não fique muito disperso no gradiente
my_optimizer = keras.optimizers.Adam(lr = 0.001, decay=0.0001, clipvalue=0.5)

# optimizer: Função para ajuste dos pesos (descida do gradiente...)
# loss: Função de perda. Como o erro será calculado
# metrics: Qual a métrica que será utilizada para avaliação
# classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])

classifier.compile(optimizer=my_optimizer, loss='binary_crossentropy', metrics=['binary_accuracy'])



# Calcular o erro para "batch_size" e depois atualizar os pesos
# epochs: Quantas épocas eu quero executar o treinamento
classifier.fit(X_train, y_train, batch_size = 10, epochs=100)

weights0 = classifier.layers[0].get_weights()
weights1 = classifier.layers[1].get_weights()

Epoch 1/100
426/426 [==============================] - 0s 775us/step - loss: 2.1389 - binary_accuracy: 0.5141
Epoch 2/100
426/426 [==============================] - 0s 220us/step - loss: 0.7424 - binary_accuracy: 0.6948
Epoch 3/100
426/426 [==============================] - 0s 340us/step - loss: 0.5423 - binary_accuracy: 0.7207
Epoch 4/100
426/426 [==============================] - ETA: 0s - loss: 0.4198 - binary_accuracy: 0.788 - 0s 381us/step - loss: 0.5048 - binary_accuracy: 0.7887
Epoch 5/100
426/426 [==============================] - 0s 338us/step - loss: 0.6763 - binary_accuracy: 0.7793
Epoch 6/100
426/426 [==============================] - 0s 244us/step - loss: 0.6504 - binary_accuracy: 0.7817
Epoch 7/100
426/426 [==============================] - 0s 318us/step - loss: 0.4693 - binary_accuracy: 0.8404
Epoch 8/100
426/426 [==============================] - 0s 211us/step - loss: 0.4425 - binary_accuracy: 0.8286
Epoch 9/100
426/426 [==============================] - 0s 223us/step -

In [6]:
predictions = classifier.predict(X_test)
predictions = (predictions > 0.5)

In [7]:
from sklearn.metrics import confusion_matrix, accuracy_score
precision = accuracy_score(y_test, predictions)
print(precision)

precision = confusion_matrix(y_test, predictions)
print(precision)


0.804195804196
[[36 16]
 [12 79]]


In [8]:
result = classifier.evaluate(X_test, y_test)
result

143/143 [==============================] - 0s 285us/step


[1.1032651439830139, 0.80419580502943555]

## Cross Validation

In [9]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
import keras

In [10]:
def build_network():
    classifier = Sequential()
    classifier.add(Dense(units=16, activation='relu', kernel_initializer='random_uniform', input_dim=30))
    classifier.add(Dense(units=16, activation='relu', kernel_initializer='random_uniform'))
    classifier.add(Dense(units=1, activation='sigmoid'))
    my_optimizer = keras.optimizers.Adam(lr = 0.001, decay=0.0001, clipvalue=0.5)
    classifier.compile(optimizer=my_optimizer, loss='binary_crossentropy', metrics=['binary_accuracy'])
    
    return classifier


In [11]:
classifier = KerasClassifier(build_fn=build_network, epochs=100, batch_size=10)

In [12]:
results = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv=10, scoring='accuracy')

Epoch 1/100
383/383 [==============================] - 0s 721us/step - loss: 0.9742 - binary_accuracy: 0.6214
Epoch 2/100
383/383 [==============================] - 0s 107us/step - loss: 0.5446 - binary_accuracy: 0.6945
Epoch 3/100
383/383 [==============================] - 0s 148us/step - loss: 0.4816 - binary_accuracy: 0.7650
Epoch 4/100
383/383 [==============================] - 0s 143us/step - loss: 0.4457 - binary_accuracy: 0.7807
Epoch 5/100
383/383 [==============================] - 0s 140us/step - loss: 0.4246 - binary_accuracy: 0.7781
Epoch 6/100
383/383 [==============================] - 0s 129us/step - loss: 0.4346 - binary_accuracy: 0.8146
Epoch 7/100
383/383 [==============================] - 0s 124us/step - loss: 0.3832 - binary_accuracy: 0.8172
Epoch 8/100
383/383 [==============================] - 0s 93us/step - loss: 0.4049 - binary_accuracy: 0.8198
Epoch 9/100
383/383 [==============================] - 0s 97us/step - loss: 0.4297 - binary_accuracy: 0.8277
Epoch 10/100

383/383 [==============================] - 0s 101us/step - loss: 0.3227 - binary_accuracy: 0.8956
Epoch 76/100
383/383 [==============================] - 0s 119us/step - loss: 0.2980 - binary_accuracy: 0.8799
Epoch 77/100
383/383 [==============================] - 0s 110us/step - loss: 0.3558 - binary_accuracy: 0.8982
Epoch 78/100
383/383 [==============================] - 0s 94us/step - loss: 0.3343 - binary_accuracy: 0.9034
Epoch 79/100
383/383 [==============================] - 0s 97us/step - loss: 0.3244 - binary_accuracy: 0.8721
Epoch 80/100
383/383 [==============================] - 0s 97us/step - loss: 0.4234 - binary_accuracy: 0.8851
Epoch 81/100
383/383 [==============================] - 0s 97us/step - loss: 0.6043 - binary_accuracy: 0.8616
Epoch 82/100
383/383 [==============================] - 0s 103us/step - loss: 0.5148 - binary_accuracy: 0.8564
Epoch 83/100
383/383 [==============================] - 0s 108us/step - loss: 0.5512 - binary_accuracy: 0.8590
Epoch 84/100
383/3

383/383 [==============================] - 0s 104us/step - loss: 1.1033 - binary_accuracy: 0.8068
Epoch 50/100
383/383 [==============================] - 0s 119us/step - loss: 0.8849 - binary_accuracy: 0.8381
Epoch 51/100
383/383 [==============================] - 0s 106us/step - loss: 0.7324 - binary_accuracy: 0.8747
Epoch 52/100
383/383 [==============================] - 0s 106us/step - loss: 0.8102 - binary_accuracy: 0.8642
Epoch 53/100
383/383 [==============================] - 0s 179us/step - loss: 0.8595 - binary_accuracy: 0.8486
Epoch 54/100
383/383 [==============================] - 0s 201us/step - loss: 0.7915 - binary_accuracy: 0.8590
Epoch 55/100
383/383 [==============================] - 0s 135us/step - loss: 0.7804 - binary_accuracy: 0.8486
Epoch 56/100
383/383 [==============================] - 0s 165us/step - loss: 0.7372 - binary_accuracy: 0.8616
Epoch 57/100
383/383 [==============================] - 0s 153us/step - loss: 0.7668 - binary_accuracy: 0.8616
Epoch 58/100
3

383/383 [==============================] - 0s 93us/step - loss: 0.3745 - binary_accuracy: 0.8668
Epoch 24/100
383/383 [==============================] - 0s 83us/step - loss: 0.5394 - binary_accuracy: 0.8120
Epoch 25/100
383/383 [==============================] - 0s 93us/step - loss: 0.4377 - binary_accuracy: 0.8695
Epoch 26/100
383/383 [==============================] - 0s 92us/step - loss: 0.3896 - binary_accuracy: 0.8329
Epoch 27/100
383/383 [==============================] - 0s 88us/step - loss: 0.3465 - binary_accuracy: 0.8851
Epoch 28/100
383/383 [==============================] - 0s 94us/step - loss: 0.4784 - binary_accuracy: 0.8068
Epoch 29/100
383/383 [==============================] - 0s 93us/step - loss: 0.5245 - binary_accuracy: 0.8407
Epoch 30/100
383/383 [==============================] - 0s 90us/step - loss: 0.4587 - binary_accuracy: 0.8433
Epoch 31/100
383/383 [==============================] - 0s 91us/step - loss: 0.3664 - binary_accuracy: 0.8903
Epoch 32/100
383/383 [=

383/383 [==============================] - 0s 100us/step - loss: 1.0390 - binary_accuracy: 0.8172
Epoch 72/100
383/383 [==============================] - 0s 92us/step - loss: 1.1263 - binary_accuracy: 0.7990
Epoch 73/100
383/383 [==============================] - 0s 93us/step - loss: 0.9643 - binary_accuracy: 0.8460
Epoch 74/100
383/383 [==============================] - 0s 94us/step - loss: 1.0132 - binary_accuracy: 0.8303
Epoch 75/100
383/383 [==============================] - 0s 92us/step - loss: 1.0195 - binary_accuracy: 0.8303
Epoch 76/100
383/383 [==============================] - 0s 97us/step - loss: 0.9333 - binary_accuracy: 0.8590
Epoch 77/100
383/383 [==============================] - 0s 93us/step - loss: 0.9543 - binary_accuracy: 0.8538
Epoch 78/100
383/383 [==============================] - 0s 93us/step - loss: 1.0537 - binary_accuracy: 0.8381
Epoch 79/100
383/383 [==============================] - 0s 92us/step - loss: 1.0309 - binary_accuracy: 0.8381
Epoch 80/100
383/383 [

383/383 [==============================] - 0s 100us/step - loss: 0.2995 - binary_accuracy: 0.8877
Epoch 20/100
383/383 [==============================] - 0s 104us/step - loss: 0.4100 - binary_accuracy: 0.8616
Epoch 21/100
383/383 [==============================] - 0s 184us/step - loss: 0.5098 - binary_accuracy: 0.8407
Epoch 22/100
383/383 [==============================] - 0s 121us/step - loss: 0.4082 - binary_accuracy: 0.8695
Epoch 23/100
383/383 [==============================] - 0s 159us/step - loss: 0.3581 - binary_accuracy: 0.8799
Epoch 24/100
383/383 [==============================] - 0s 140us/step - loss: 0.3299 - binary_accuracy: 0.8851
Epoch 25/100
383/383 [==============================] - 0s 127us/step - loss: 0.3894 - binary_accuracy: 0.8747
Epoch 26/100
383/383 [==============================] - 0s 121us/step - loss: 0.3644 - binary_accuracy: 0.8695
Epoch 27/100
383/383 [==============================] - 0s 132us/step - loss: 0.4667 - binary_accuracy: 0.8538
Epoch 28/100
3

384/384 [==============================] - 0s 99us/step - loss: 1.0206 - binary_accuracy: 0.8385
Epoch 66/100
384/384 [==============================] - 0s 105us/step - loss: 1.0234 - binary_accuracy: 0.8203
Epoch 67/100
384/384 [==============================] - 0s 94us/step - loss: 1.1733 - binary_accuracy: 0.7891
Epoch 68/100
384/384 [==============================] - 0s 102us/step - loss: 0.8896 - binary_accuracy: 0.8516
Epoch 69/100
384/384 [==============================] - 0s 101us/step - loss: 1.1763 - binary_accuracy: 0.8229
Epoch 70/100
384/384 [==============================] - 0s 106us/step - loss: 0.9690 - binary_accuracy: 0.8411
Epoch 71/100
384/384 [==============================] - 0s 107us/step - loss: 1.0185 - binary_accuracy: 0.8464
Epoch 72/100
384/384 [==============================] - 0s 115us/step - loss: 1.0927 - binary_accuracy: 0.8229
Epoch 73/100
384/384 [==============================] - 0s 102us/step - loss: 1.2406 - binary_accuracy: 0.8281
Epoch 74/100
384

384/384 [==============================] - 0s 99us/step - loss: 0.3799 - binary_accuracy: 0.8672
Epoch 14/100
384/384 [==============================] - 0s 97us/step - loss: 0.3690 - binary_accuracy: 0.8594
Epoch 15/100
384/384 [==============================] - 0s 99us/step - loss: 0.5031 - binary_accuracy: 0.8542
Epoch 16/100
384/384 [==============================] - 0s 96us/step - loss: 0.3955 - binary_accuracy: 0.8672
Epoch 17/100
384/384 [==============================] - 0s 100us/step - loss: 0.3881 - binary_accuracy: 0.8516
Epoch 18/100
384/384 [==============================] - 0s 100us/step - loss: 0.3918 - binary_accuracy: 0.8776
Epoch 19/100
384/384 [==============================] - 0s 98us/step - loss: 0.3009 - binary_accuracy: 0.8958
Epoch 20/100
384/384 [==============================] - 0s 99us/step - loss: 0.3375 - binary_accuracy: 0.8958
Epoch 21/100
384/384 [==============================] - 0s 100us/step - loss: 0.5069 - binary_accuracy: 0.8411
Epoch 22/100
384/384

384/384 [==============================] - 0s 117us/step - loss: 0.3152 - binary_accuracy: 0.8906
Epoch 62/100
384/384 [==============================] - 0s 96us/step - loss: 0.3614 - binary_accuracy: 0.8750
Epoch 63/100
384/384 [==============================] - 0s 99us/step - loss: 0.3024 - binary_accuracy: 0.9010
Epoch 64/100
384/384 [==============================] - 0s 101us/step - loss: 0.2955 - binary_accuracy: 0.8854
Epoch 65/100
384/384 [==============================] - 0s 107us/step - loss: 0.3432 - binary_accuracy: 0.8750
Epoch 66/100
384/384 [==============================] - 0s 104us/step - loss: 0.3388 - binary_accuracy: 0.8906
Epoch 67/100
384/384 [==============================] - 0s 104us/step - loss: 0.3840 - binary_accuracy: 0.8724
Epoch 68/100
384/384 [==============================] - 0s 105us/step - loss: 0.3170 - binary_accuracy: 0.8906
Epoch 69/100
384/384 [==============================] - 0s 104us/step - loss: 0.3447 - binary_accuracy: 0.8958
Epoch 70/100
384

In [13]:
results.mean()
results.std()

0.051031055082987914

# Dropout

Zera alguns neurônios das camadas para reduzir o overffiting


In [14]:
from keras.layers import Dropout

def build_network_with_dropout():
    classifier = Sequential()
    classifier.add(Dense(units=16, activation='relu', kernel_initializer='random_uniform', input_dim=30))
    classifier.add(Dropout(0.2))
    classifier.add(Dense(units=16, activation='relu', kernel_initializer='random_uniform'))
    classifier.add(Dropout(0.2))
    classifier.add(Dense(units=1, activation='sigmoid'))
    my_optimizer = keras.optimizers.Adam(lr = 0.001, decay=0.0001, clipvalue=0.5)
    classifier.compile(optimizer=my_optimizer, loss='binary_crossentropy', metrics=['binary_accuracy'])
    
    return classifier

# Tuning

Quantos neurônios na camada escondida? Qual loss function vamos utilizar? Como fazer a incialização dos pesos? Qual função de ativação?

In [15]:
from sklearn.model_selection import GridSearchCV

In [16]:
def build_custom_network(optimizer, loss, kernel_initializer, activation, neurons):
    classifier = Sequential()
    classifier.add(Dense(units=neurons, activation=activation, kernel_initializer=kernel_initializer, input_dim=30))
    classifier.add(Dropout(0.2))
    classifier.add(Dense(units=neurons, activation=activation, kernel_initializer=kernel_initializer))
    classifier.add(Dropout(0.2))
    classifier.add(Dense(units=1, activation='sigmoid'))
    classifier.compile(optimizer=optimizer, loss=loss, metrics=['binary_accuracy'])
    
    return classifier

In [17]:
classifier = KerasClassifier(build_fn=build_custom_network)

In [27]:
parameters = {'batch_size':[10,30], 
              'epochs':[50, 100], 
              'optimizer':['adam','sgd'], 
              'loss':['binary_crossentropy', 'hinge'], 
              'kernel_initializer': ['random_uniform','normal'],
              'activation': ['relu', 'tanh'],
              'neurons': [16,8]
             }

In [28]:
grid_search = GridSearchCV(estimator=classifier, param_grid=parameters, scoring='accuracy', cv=5)
grid_search = grid_search.fit(X_train, y_train)
best_parameters =  grid_search.best_params_
best_score = grid_search.best_score_

Epoch 1/50
340/340 [==============================] - 1s 2ms/step - loss: 1.9268 - binary_accuracy: 0.4882
Epoch 2/50
340/340 [==============================] - 0s 124us/step - loss: 0.6025 - binary_accuracy: 0.6235
Epoch 3/50
340/340 [==============================] - 0s 132us/step - loss: 0.5747 - binary_accuracy: 0.6382
Epoch 4/50
340/340 [==============================] - 0s 134us/step - loss: 0.5121 - binary_accuracy: 0.6971
Epoch 5/50
340/340 [==============================] - 0s 125us/step - loss: 0.5088 - binary_accuracy: 0.6912
Epoch 6/50
340/340 [==============================] - 0s 128us/step - loss: 0.4900 - binary_accuracy: 0.7353
Epoch 7/50
340/340 [==============================] - 0s 124us/step - loss: 0.5018 - binary_accuracy: 0.7176
Epoch 8/50
340/340 [==============================] - 0s 121us/step - loss: 0.4841 - binary_accuracy: 0.7382
Epoch 9/50
340/340 [==============================] - 0s 128us/step - loss: 0.4798 - binary_accuracy: 0.7500
Epoch 10/50
340/340 [

341/341 [==============================] - 0s 192us/step - loss: 0.3214 - binary_accuracy: 0.8739
Epoch 50/50
341/341 [==============================] - 0s 187us/step - loss: 0.3161 - binary_accuracy: 0.8680
Epoch 1/50
341/341 [==============================] - 1s 4ms/step - loss: 0.9517 - binary_accuracy: 0.6158
Epoch 2/50
341/341 [==============================] - 0s 172us/step - loss: 0.6696 - binary_accuracy: 0.6334
Epoch 3/50
341/341 [==============================] - 0s 180us/step - loss: 0.5964 - binary_accuracy: 0.6657
Epoch 4/50
341/341 [==============================] - 0s 197us/step - loss: 0.5012 - binary_accuracy: 0.7243
Epoch 5/50
341/341 [==============================] - 0s 181us/step - loss: 0.5194 - binary_accuracy: 0.7419
Epoch 6/50
341/341 [==============================] - 0s 208us/step - loss: 0.4576 - binary_accuracy: 0.7654
Epoch 7/50
341/341 [==============================] - 0s 169us/step - loss: 0.4883 - binary_accuracy: 0.7771
Epoch 8/50
341/341 [===========

340/340 [==============================] - 0s 363us/step - loss: 4.8472 - binary_accuracy: 0.6059
Epoch 48/50
340/340 [==============================] - 0s 337us/step - loss: 4.7180 - binary_accuracy: 0.6059
Epoch 49/50
340/340 [==============================] - 0s 551us/step - loss: 5.0298 - binary_accuracy: 0.6059
Epoch 50/50
340/340 [==============================] - 0s 437us/step - loss: 5.2366 - binary_accuracy: 0.6059
Epoch 1/50
341/341 [==============================] - 2s 6ms/step - loss: 6.0710 - binary_accuracy: 0.6070
Epoch 2/50
341/341 [==============================] - 0s 476us/step - loss: 6.1593 - binary_accuracy: 0.6129
Epoch 3/50
341/341 [==============================] - 0s 457us/step - loss: 6.1245 - binary_accuracy: 0.6158
Epoch 4/50
341/341 [==============================] - 0s 423us/step - loss: 6.1245 - binary_accuracy: 0.6158
Epoch 5/50
341/341 [==============================] - 0s 529us/step - loss: 6.1245 - binary_accuracy: 0.6158
Epoch 6/50
341/341 [=========

341/341 [==============================] - 0s 147us/step - loss: 9.1645 - binary_accuracy: 0.3988
Epoch 46/50
341/341 [==============================] - 0s 138us/step - loss: 9.1551 - binary_accuracy: 0.4106
Epoch 47/50
341/341 [==============================] - 0s 149us/step - loss: 8.8799 - binary_accuracy: 0.4282
Epoch 48/50
341/341 [==============================] - 0s 171us/step - loss: 8.9721 - binary_accuracy: 0.4076
Epoch 49/50
341/341 [==============================] - 0s 176us/step - loss: 8.3861 - binary_accuracy: 0.4457
Epoch 50/50
341/341 [==============================] - 0s 247us/step - loss: 8.5028 - binary_accuracy: 0.4516
Epoch 1/50
341/341 [==============================] - 1s 4ms/step - loss: 5.9914 - binary_accuracy: 0.6070
Epoch 2/50
341/341 [==============================] - 0s 247us/step - loss: 5.8589 - binary_accuracy: 0.6305
Epoch 3/50
341/341 [==============================] - 0s 266us/step - loss: 5.7755 - binary_accuracy: 0.6334
Epoch 4/50
341/341 [=======

341/341 [==============================] - 0s 132us/step - loss: 0.3676 - binary_accuracy: 0.8299
Epoch 44/50
341/341 [==============================] - 0s 170us/step - loss: 0.3697 - binary_accuracy: 0.8270
Epoch 45/50
341/341 [==============================] - 0s 176us/step - loss: 0.3552 - binary_accuracy: 0.8358
Epoch 46/50
341/341 [==============================] - 0s 229us/step - loss: 0.3903 - binary_accuracy: 0.8152
Epoch 47/50
341/341 [==============================] - 0s 164us/step - loss: 0.3631 - binary_accuracy: 0.8299
Epoch 48/50
341/341 [==============================] - 0s 154us/step - loss: 0.3589 - binary_accuracy: 0.8328
Epoch 49/50
341/341 [==============================] - 0s 153us/step - loss: 0.3474 - binary_accuracy: 0.8475
Epoch 50/50
341/341 [==============================] - 0s 150us/step - loss: 0.3893 - binary_accuracy: 0.8123
Epoch 1/50
341/341 [==============================] - 2s 5ms/step - loss: 2.2123 - binary_accuracy: 0.3930
Epoch 2/50
341/341 [=====

341/341 [==============================] - 0s 132us/step - loss: 0.2576 - binary_accuracy: 0.9062
Epoch 42/50
341/341 [==============================] - 0s 150us/step - loss: 0.2556 - binary_accuracy: 0.9091
Epoch 43/50
341/341 [==============================] - 0s 133us/step - loss: 0.3204 - binary_accuracy: 0.8651
Epoch 44/50
341/341 [==============================] - 0s 135us/step - loss: 0.2564 - binary_accuracy: 0.9062
Epoch 45/50
341/341 [==============================] - 0s 137us/step - loss: 0.2531 - binary_accuracy: 0.9091
Epoch 46/50
341/341 [==============================] - 0s 139us/step - loss: 0.2514 - binary_accuracy: 0.9179
Epoch 47/50
341/341 [==============================] - 0s 142us/step - loss: 0.2708 - binary_accuracy: 0.9032
Epoch 48/50
341/341 [==============================] - 0s 136us/step - loss: 0.2320 - binary_accuracy: 0.9208
Epoch 49/50
341/341 [==============================] - 0s 140us/step - loss: 0.2513 - binary_accuracy: 0.9208
Epoch 50/50
341/341 [=

341/341 [==============================] - 0s 417us/step - loss: 0.6132 - binary_accuracy: 0.6246
Epoch 40/50
341/341 [==============================] - 0s 314us/step - loss: 0.6116 - binary_accuracy: 0.6246
Epoch 41/50
341/341 [==============================] - 0s 310us/step - loss: 0.6164 - binary_accuracy: 0.6246
Epoch 42/50
341/341 [==============================] - 0s 309us/step - loss: 0.6118 - binary_accuracy: 0.6246
Epoch 43/50
341/341 [==============================] - 0s 380us/step - loss: 0.6132 - binary_accuracy: 0.6246
Epoch 44/50
341/341 [==============================] - 0s 352us/step - loss: 0.6133 - binary_accuracy: 0.6246
Epoch 45/50
341/341 [==============================] - 0s 379us/step - loss: 0.6163 - binary_accuracy: 0.6246
Epoch 46/50
341/341 [==============================] - 0s 400us/step - loss: 0.6148 - binary_accuracy: 0.6246
Epoch 47/50
341/341 [==============================] - 0s 346us/step - loss: 0.6133 - binary_accuracy: 0.6246
Epoch 48/50
341/341 [=

341/341 [==============================] - 0s 516us/step - loss: 0.6945 - binary_accuracy: 0.6364
Epoch 14/50
341/341 [==============================] - 0s 488us/step - loss: 0.7279 - binary_accuracy: 0.6364
Epoch 15/50
341/341 [==============================] - 0s 462us/step - loss: 0.6942 - binary_accuracy: 0.6364
Epoch 16/50
341/341 [==============================] - 0s 451us/step - loss: 0.7370 - binary_accuracy: 0.6364
Epoch 17/50
341/341 [==============================] - 0s 488us/step - loss: 0.6873 - binary_accuracy: 0.6364
Epoch 18/50
341/341 [==============================] - 0s 449us/step - loss: 0.7328 - binary_accuracy: 0.6364
Epoch 19/50
341/341 [==============================] - 0s 277us/step - loss: 0.6903 - binary_accuracy: 0.6364
Epoch 20/50
341/341 [==============================] - 0s 302us/step - loss: 0.7307 - binary_accuracy: 0.6364
Epoch 21/50
341/341 [==============================] - 0s 360us/step - loss: 0.6438 - binary_accuracy: 0.6364
Epoch 22/50
341/341 [=

KeyboardInterrupt: 

In [32]:
classifier = build_custom_network(optimizer='adam', loss='binary_crossentropy', kernel_initializer='normal', activation='relu', neurons=8)
classifier.fit(X_train, y_train, batch_size=10, epochs=100)

Epoch 1/100
426/426 [==============================] - 3s 8ms/step - loss: 4.0420 - binary_accuracy: 0.5305
Epoch 2/100
426/426 [==============================] - 0s 479us/step - loss: 1.6443 - binary_accuracy: 0.6033
Epoch 3/100
426/426 [==============================] - 0s 451us/step - loss: 0.9542 - binary_accuracy: 0.6009
Epoch 4/100
426/426 [==============================] - 0s 439us/step - loss: 0.6912 - binary_accuracy: 0.6385
Epoch 5/100
426/426 [==============================] - 0s 534us/step - loss: 0.6847 - binary_accuracy: 0.5962
Epoch 6/100
426/426 [==============================] - 0s 321us/step - loss: 0.6046 - binary_accuracy: 0.6127
Epoch 7/100
426/426 [==============================] - 0s 294us/step - loss: 0.5913 - binary_accuracy: 0.6643
Epoch 8/100
426/426 [==============================] - 0s 344us/step - loss: 0.6087 - binary_accuracy: 0.6103
Epoch 9/100
426/426 [==============================] - 0s 302us/step - loss: 0.5571 - binary_accuracy: 0.6408
Epoch 10/100

## Classifying

In [39]:
import numpy as np 
new_instance = np.array([[15.80, 8.34, 118, 900, 0.10, 0.26, 0.08, 0.134, 0.178,
                  0.20, 0.05, 1098, 0.87, 4500, 145.2, 0.005, 0.04, 0.05, 0.015,
                  0.03, 0.007, 23.15, 16.64, 178.5, 2018, 0.14, 0.185,
                  0.84, 158, 0.363]])
prediction = classifier.predict(new_instance)
prediction = (prediction > 0.5)
prediction

array([[ True]], dtype=bool)

## Saving Model

In [40]:
# classifier_json = classifier_to_json()
# with open('binary_classifier','w') as json_file:
#     json_file.write(classifier_json)
# classifier.save_weights('classifier_weights.h5')

## Loading Model

In [42]:
from keras.models import model_from_json

In [ ]:
# my_file = open('binary_classifier','r')
# network = my.file.read()
# my_file.close()

# classifier = model_from_json(network)
# classifier.load_weights('classifier_weights.h5')